In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz, DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingRegressor
from IPython.display import Image
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.read_csv('houses_to_rent_v2.csv')

In [3]:
df.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
0,São Paulo,70,2,1,1,7,acept,furnished,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750,63,7973
2,Porto Alegre,80,1,1,1,6,acept,not furnished,1000,2800,0,41,3841
3,Porto Alegre,51,2,1,0,2,acept,not furnished,270,1112,22,17,1421
4,São Paulo,25,1,1,0,1,not acept,not furnished,0,800,25,11,836


In [4]:
df.drop(['city'], axis=1, inplace=True)

In [5]:
df.drop(['floor'], axis=1, inplace=True)

In [6]:
df.drop(['fire insurance (R$)'], axis=1, inplace=True)

In [7]:
df.drop(['animal'], axis=1, inplace=True)

In [8]:
df.drop(['furniture'], axis=1, inplace=True)

In [9]:
df.drop(['parking spaces'], axis=1, inplace=True)

In [10]:
df['total (R$)'].mean()

5490.4869996258885

In [11]:
df.sort_values(by=['total (R$)'], inplace=True)

In [12]:
df.head(7290)

,area,rooms,bathroom,hoa (R$),rent amount (R$),property tax (R$),total (R$)
10627,22,1,1,30,450,13,499
5876,30,1,1,0,500,0,507
9317,30,1,1,0,500,0,507
5152,35,1,1,0,500,0,508
8183,30,1,1,0,500,0,509
...,...,...,...,...,...,...,...
10070,320,4,5,0,5000,409,5485
5342,140,3,3,2285,2820,343,5485
1985,142,2,3,1400,3550,492,5487
8350,215,3,2,1400,4000,38,5490


In [13]:
accuracy, global_accuracy = {}, {} 

In [14]:
def add_stat(name, model):
    
    accuracy[name] = accuracy_score(model.predict(X_test), y_test)
    global_accuracy[name] = accuracy_score(model.predict(X_test), y_test)

In [15]:
y = df['total (R$)']
X = df.drop(['total (R$)'], axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2, random_state=17)

In [17]:
model = DecisionTreeClassifier(max_depth=6, random_state=17)

params = {'max_depth': np.arange(1, 6),'min_samples_leaf':np.arange(10, 20), 'max_features': np.arange(5, 7)}

grid_search = GridSearchCV(model, params, n_jobs=-1, cv=2, return_train_score=True)
grid_search.fit(X_train, y_train)

c:\users\user\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\model_selection\_split.py:665: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=2, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=6,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=17,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': array([1, 2, 3, 4, 5]),
                       

In [18]:
grid_search.best_params_, grid_search.best_score_

({'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 10},
 0.01674462114125351)

In [19]:
decision_tree = DecisionTreeClassifier(max_depth=5,
                                       max_features=6,
                                       min_samples_leaf=10)

In [20]:
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=6, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [21]:
add_stat('Decision Tree', decision_tree)

In [22]:
graph = DecisionTreeClassifier(max_depth=6)
graph.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [23]:
export_graphviz(graph, feature_names=df.columns[:0-1], out_file='Decision Tree.dot', filled=True)

In [24]:
!dot -Tpng 'Decision Tree.dot' -o 'Decision Tree.png'

"dot" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [25]:
Image(filename='Decision_Tree.png')

FileNotFoundError: [Errno 2] No such file or directory: 'Decision Tree.png'

In [0]:
linear = LinearRegression(n_jobs=-1).fit(X_train, y_train)

In [44]:
pred = linear.predict(X_test)
line_acc = accuracy_score(np.around(pred), y_test) 
line_acc

0.0

In [0]:
accuracy['Linear Regression'] = accuracy_score(np.around(linear.predict(X_test)), y_test)
global_accuracy['Linear Regression'] = accuracy_score(np.around(linear.predict(X)), y)

In [46]:
model = KNeighborsClassifier(n_jobs=-1)

params = {'n_neighbors': np.arange(1, 7)}

grid_search = GridSearchCV(model, params, n_jobs=-1, return_train_score=True)

grid_search.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_neighbors': array([1, 2, 3, 4, 5, 6])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [47]:
grid_search.best_params_, grid_search.best_score_

({'n_neighbors': 1}, 0.10757717492984095)

In [0]:
neighbour = KNeighborsClassifier(n_jobs=-1, n_neighbors=5).fit(X_train, y_train)

In [0]:
add_stat("KNEighbors", neighbour)

In [50]:
model = RandomForestClassifier(n_jobs=-1,
                              random_state=17)

params = {
    'max_depth': np.arange(1, 3),
    'min_samples_leaf' : np.arange(10, 20),
    'max_features' : np.arange(2, 6),
    'n_estimators': np.arange(1, 3)
}

grid_search = GridSearchCV(model, params, 
                           n_jobs=-1, cv=3,
                           return_train_score=True)

grid_search.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False, random_state=17,
                                    

In [ ]:
grid_search.best_params_, grid_search.best_score_

In [73]:
random_forest = RandomForestClassifier(n_jobs=-1,
                                       random_state=42,
                                       max_depth=10,
                                       max_features=3,
                                       min_samples_leaf=1,
                                       n_estimators=4)

random_forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4, n_jobs=-1,
                       oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
model = GradientBoostingClassifier(random_state=17)
params = {'n_estimators': np.arange(50,60), 'max_depth': np.arange(1,4), 'min_samples_leaf': np.arange(1,10), 
           'max_features': np.arange(1, 6)}

grid_search = GridSearchCV(model, params, n_jobs=-1, return_train_score=True)

grid_search.fit(X_train, y_train)

In [0]:
model = LogisticRegression(random_state=17,
                          n_jobs=1)

params = {
    'C': np.linspace(1, 100)
}

grid_search = GridSearchCV(model, params, 
                           n_jobs=1,
                           return_train_score=True)

grid_search.fit(X_train, y_train)